## Sentinel-3 and Sentinel-5p Stactools Packages

### Objective

Update the existing [sentinel3](https://github.com/stactools-packages/sentinel3) and [sentinel5p](https://github.com/stactools-packages/sentinel5p) stactools packages to produce STAC Items matching those accessible from the Planetary Computer STAC [API](https://planetarycomputer.microsoft.com/api/stac/v1). The STAC Items describe NetCDF assets.

### Background

The Sentinel-3 [data](https://github.com/microsoft/AIforEarthDataSets/blob/main/data/sentinel-3.md) is divided by product into [12 STAC Collections](https://planetarycomputer.microsoft.com/dataset/group/sentinel-3). We'll need to create STAC Items for assets from each Collection.
- [sentinel-3-olci-wfr-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-olci-wfr-l2-netcdf)
- [sentinel-3-olci-lfr-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-olci-lfr-l2-netcdf)
- [sentinel-3-slstr-frd-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-slstr-frp-l2-netcdf)
- [sentinel-3-slstr-lst-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-slstr-lst-l2-netcdf)
- [sentinel-3-slstr-wst-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-slstr-wst-l2-netcdf)
- [sentinel-3-sral-lan-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-sral-lan-l2-netcdf)
- [sentinel-3-sral-wat-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-sral-wat-l2-netcdf)
- [sentinel-3-synergy-aod-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-synergy-aod-l2-netcdf)
- [sentinel-3-synergy-syn-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-synergy-syn-l2-netcdf)
- [sentinel-3-synergy-v10-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-synergy-v10-l2-netcdf)
- [sentinel-3-synergy-vg1-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-synergy-vg1-l2-netcdf)
- [sentinel-3-synergy-vgp-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-3-synergy-vgp-l2-netcdf)

The Sentinel-5p [data](https://github.com/microsoft/AIforEarthDataSets/blob/main/data/sentinel-5p.md) is aggregated into [one STAC Collection](https://planetarycomputer.microsoft.com/dataset/sentinel-5p-l2-netcdf) ([sentinel-5p-l2-netcdf](https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-5p-l2-netcdf)), but consists of 13 different products. We'll need to create STAC Items for each product.
- `L2__AER_AI`: Ultraviolet aerosol index
- `L2__AER_LH`: Aerosol layer height
- `L2__CH4___`: Methane (CH<sub>4</sub>) total column
- `L2__CLOUD_`: Cloud fraction, albedo, and top pressure
- `L2__CO____`: Carbon monoxide (CO) total column
- `L2__HCHO__`: Formaldehyde (HCHO) total column
- `L2__NO2___`: Nitrogen dioxide (NO<sub>2</sub>) total column
- `L2__O3____`: Ozone (O<sub>3</sub>) total column
- `L2__O3_TCL`: Ozone (O<sub>3</sub>) tropospheric column
- `L2__SO2___`: Sulfur dioxide (SO<sub>2</sub>) total column
- `L2__NP_BD3`: Cloud from the Suomi NPP mission, band 3
- `L2__NP_BD6`: Cloud from the Suomi NPP mission, band 6
- `L2__NP_BD7`: Cloud from the Suomi NPP mission, band 7

STAC Item details:
- Per Microsoft's direction, Sentinel-3 and Sentinel-5p STAC Items were derived from STAC Items provided by [Sinergise](https://www.sinergise.com/) to Microsoft.
- The Sinergise STAC Items only exist in Azure blob storage. They were never ingested into a database or made available via a STAC API.
- The Sinergise STAC Items are not fully featured and do not follow best practices. Consequently, the derived STAC Items may feel awkward or deficient in certain aspects.
- The code that transforms Sinergise STAC Items into the Items on the Planetary Computer is public on GitHub:
    - Sentinel-3: https://github.com/microsoft/planetary-computer-tasks/tree/main/datasets/sentinel-3/sentinel_3
    - Sentinel-5p: https://github.com/microsoft/planetary-computer-tasks/blob/main/datasets/sentinel-5p/sentinel_5p.py


### Accessing STAC Items and NetCDF assets

You'll likely want some example STAC and corresponding NetCDF assets. See below for an example of one way to go about that. You may also find [Pete's Planetary Computer downloader](https://github.com/gadomski/pc-rs) useful, but you'll need some Item IDs to use it.

In [ ]:
import json
import os

import planetary_computer as pc
import pystac_client
import requests

#### Sentinel-3

In [5]:
# get an item and save it to file
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace
)
search = catalog.search(
    collections="sentinel-3-olci-wfr-l2-netcdf",  # this is just one of 12 possible collections
    query={
        "s3:processing_timeliness": {"eq": "NT"},  # There are also "ST" and "NR" products
    },
    limit=1
)
item = next(search.get_items())

os.mkdir(item.id)
with open(os.path.join(item.id, "item.json"), "w") as f:
    f.write(json.dumps(item.to_dict(), indent=2))



In [6]:
# download the item assets
for asset in item.assets.values():
    file_name = asset.href.split("?")[0].split("/")[-1]
    local_path = os.path.join(item.id, file_name)
    with requests.get(asset.href, stream=True) as r:
        r.raise_for_status()
        with open(local_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

#### Sentinel-5p

In [ ]:
# get an item and save it to file
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace
)
search = catalog.search(
    collections="sentinel-5p-l2-netcdf",
    query={
        "s5p:product_type": {"eq": "L2__AER_AI"},  # this is just one of 13 product types
        "s5p:processing_mode": {"eq": "OFFL"},  # there are also "NRTI" and "RPRO" products
    },
    limit=1
)
item = next(search.get_items())

os.mkdir(item.id)
with open(os.path.join(item.id, "item.json"), "w") as f:
    f.write(json.dumps(item.to_dict(), indent=2))

In [ ]:
# download the item assets
for asset in item.assets.values():
    file_name = asset.href.split("?")[0].split("/")[-1]
    local_path = os.path.join(item.id, file_name)
    with requests.get(asset.href, stream=True) as r:
        r.raise_for_status()
        with open(local_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)